## Fully-connected Network

In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets, models


In [2]:
(x, y), (test_x, test_y) = datasets.mnist.load_data()
print("X.shape", x.shape)
print("Y.len", len(y))

In [58]:
#del mnist_dataset

def mnist_dataset():
    (x, y), (test_x, test_y) = datasets.mnist.load_data()
    ds = tf.data.Dataset.from_tensor_slices((x,y))
    ds = ds.map(prepare_mnist_features_and_labels)
    ds = ds.take(20000).shuffle(20000).batch(100)
    # 20000 // 100 = 200 steps
    return ds
def prepare_mnist_features_and_labels(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    y = tf.cast(y, tf.int64)
    return x, y

In [73]:
#del model

model = models.Sequential()
model.add(layers.Input(shape=(28*28,),name="inputs"))
model.add(layers.Reshape(target_shape=(28*28,),name="reshape"))
model.add(layers.Dense(units=100, activation="relu",name="hidden_1"))
model.add(layers.Dense(units=10, activation="softmax",name="outputs"))
model.summary()

optimizer = optimizers.Adam()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 784)               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 100)               78500     
_________________________________________________________________
outputs (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [74]:

@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
      tf.nn.sparse_softmax_cross_entropy_with_logits(
          logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    accuracy = compute_accuracy(logits, y)
    
    return loss, accuracy

def train(epoch, model, optimizer):
    train_ds = mnist_dataset()
    loss = 0.0
    accuracy = 0.0
    total_step = 0
    for step, (x, y) in enumerate(train_ds):
        total_step  += 1
        loss, accuracy = train_one_step(model, optimizer, x, y)
        if step % 100 == 0:
            print('epoch', epoch,"step",step, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
    print("total_step",total_step)
    return loss, accuracy

for epoch in range(3):
    loss, accuracy = train(epoch, model, optimizer)

print('Final epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
    

epoch 0 step 0 : loss 2.3005936 ; accuracy 0.07
epoch 0 step 100 : loss 1.7287078 ; accuracy 0.77
total_step 200
epoch 1 step 0 : loss 1.6174675 ; accuracy 0.87
epoch 1 step 100 : loss 1.6210437 ; accuracy 0.85
total_step 200
epoch 2 step 0 : loss 1.5749257 ; accuracy 0.9
epoch 2 step 100 : loss 1.6490277 ; accuracy 0.82
total_step 200
Final epoch 2 : loss 1.62651 ; accuracy 0.84


In [87]:
a = model.get_layer(index=0)
print(a)
print(a.name)

reshape


In [92]:
b =  model.get_layer(index=-1)
print(b)
print(b.name)
print(b.weights)
#print(b.get_weights())

outputs
[<tf.Variable 'outputs/kernel:0' shape=(100, 10) dtype=float32, numpy=
array([[ 8.34248364e-02, -1.72429174e-01,  1.56239182e-01,
        -2.38865495e-01,  1.76677201e-02,  3.45150493e-02,
        -2.52274781e-01,  2.82926828e-01, -1.21281549e-01,
         2.79644936e-01],
       [-1.59872219e-01,  2.02070937e-01,  2.05055997e-01,
        -2.97760628e-02,  1.98996942e-02, -2.15428755e-01,
        -1.96234390e-01,  1.00327861e-02, -1.11468825e-02,
         1.52636200e-01],
       [-2.10765645e-01, -7.09046200e-02,  2.36054942e-01,
         8.56061652e-03, -3.03742409e-01,  6.99233115e-02,
        -2.38296211e-01,  2.84598351e-01,  1.31932154e-01,
        -2.06605524e-01],
       [-1.11204639e-01, -1.57788873e-01,  2.12973684e-01,
        -2.68001467e-01, -5.18870028e-03,  2.00105444e-01,
         2.82589883e-01, -3.47478151e-01,  7.82060996e-03,
        -2.23970950e-01],
       [ 2.48905599e-01,  5.39130569e-02,  2.83101201e-01,
        -1.17905475e-01, -1.98223367e-01,  2.24057

In [93]:
c =  model.get_layer(name="hidden_1")
print(c)
print(c.name)
print(c.weights)

hidden_1
[<tf.Variable 'hidden_1/kernel:0' shape=(784, 100) dtype=float32, numpy=
array([[ 0.02546474, -0.08237204,  0.06117485, ...,  0.02556056,
        -0.03219663, -0.0452243 ],
       [-0.04182609, -0.07804626, -0.07908237, ..., -0.04702512,
        -0.04411228,  0.01595496],
       [ 0.06513439, -0.03879387,  0.06709085, ...,  0.07171105,
         0.07794648, -0.01215664],
       ...,
       [ 0.02512117, -0.00476527,  0.03610925, ...,  0.05364704,
        -0.02451732,  0.06193306],
       [ 0.01317362,  0.06668244, -0.02288891, ..., -0.04366214,
        -0.05780857,  0.07380165],
       [-0.04060144, -0.05119333, -0.07450252, ..., -0.06136742,
        -0.00948126,  0.04174628]], dtype=float32)>, <tf.Variable 'hidden_1/bias:0' shape=(100,) dtype=float32, numpy=
array([ 0.04504354,  0.03513762,  0.05211701,  0.06403755,  0.02089194,
        0.02289964,  0.05137622,  0.09662849,  0.00040375,  0.01451398,
        0.07557838,  0.06295748,  0.0338307 ,  0.05768019,  0.00141871,
      

In [78]:
model.get_config()

{'name': 'sequential_16',
 'layers': [{'class_name': 'Reshape',
   'config': {'name': 'reshape',
    'trainable': True,
    'dtype': 'float32',
    'target_shape': (784,),
    'batch_input_shape': (None, 784)}},
  {'class_name': 'Dense',
   'config': {'name': 'hidden_1',
    'trainable': True,
    'dtype': 'float32',
    'units': 100,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'outputs',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': '

## Convolutional Network

In [94]:
import os, time
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, optimizers, metrics

from tensorflow.python.ops import summary_ops_v2

In [95]:
# del mnist_dataset
def mnist_dataset():
    (x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
    print("x_train.shape",x_train.shape)
    print("x_test.shape",x_test.shape)
    print("y_train.len",len(y_train))
    x_train, x_test = x_train.astype(np.float32) / 255.0, x_test.astype(np.float32) / 255.0
    y_train, y_test = y_train.astype(np.int64), y_test.astype(np.int64)
    train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test))
    return train_ds, test_ds

In [96]:
train_ds, test_ds = mnist_dataset()
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

x_train.shape (60000, 28, 28)
x_test.shape (10000, 28, 28)
y_train.len 60000


In [108]:
#del model
#del optimizer

model = models.Sequential()
model.add(layers.Input(shape=(28,28),name="inputs"))
model.add(layers.Reshape(target_shape=(28,28,1), name="reshape_1"))
model.add(layers.Conv2D(filters=2, kernel_size=5, padding="same", activation="relu", name="conv2d_1"))
model.add(layers.MaxPooling2D(pool_size=(5,5), strides=(2,2), padding="same", name="pool2d_1"))
model.add(layers.Conv2D(filters=4, kernel_size=5, padding="same", activation="relu", name="conv2d_2"))
model.add(layers.MaxPooling2D(pool_size=(5,5), strides=(2,2), padding="same", name="pool2d_2"))
model.add(layers.Flatten(name="flatten"))
model.add(layers.Dense(units=32, activation="relu", name="hidden_1"))
model.add(layers.Dropout(rate=0.3, name="dropout_1"))
model.add(layers.Dense(units=10, activation="softmax", name="outputs"))

model.summary()

optimizer = optimizers.SGD(learning_rate=0.01, momentum=0.5)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 2)         52        
_________________________________________________________________
pool2d_1 (MaxPooling2D)      (None, 14, 14, 2)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 4)         204       
_________________________________________________________________
pool2d_2 (MaxPooling2D)      (None, 7, 7, 4)           0         
_________________________________________________________________
flatten (Flatten)            (None, 196)               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 32)              

In [186]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

def train_step(model, optimizrer, images, labels):
    with tf.GradientTape() as tape:
        logits = model(images, training=True)
        loss = compute_loss(labels, logits)
        compute_accuracy(labels, logits)
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

def train(model, optimizer, dataset, log_freq=100):    
    avg_loss = metrics.Mean("loss", dtype=tf.float32)
    for images, labels in dataset:
        loss = train_step(model, optimizer, images, labels)
        avg_loss(loss)
        
        if tf.equal(optimizer.iterations % log_freq, 0):
            print("step:", int(optimizer.iterations),
                  "loss:", avg_loss.result().numpy(),
                  "acc:", compute_accuracy.result().numpy())
            avg_loss.reset_states()
            compute_accuracy.reset_states()
            
def test(model, dataset, step_num):
    avg_loss = metrics.Mean("loss", dtype=tf.float32)
    for (images, labels) in dataset:
        logits = model(images, training=False)
        avg_loss(compute_loss(labels, logits))
        compute_accuracy(labels, logits)
        
    print('Model test set loss: {:0.4f} accuracy: {:0.2f}%'.format(
        avg_loss.result(), compute_accuracy.result() * 100))
    print('loss:', avg_loss.result(), 'acc:', compute_accuracy.result())
    summary_ops_v2.scalar('loss', avg_loss.result(), step=step_num)
    summary_ops_v2.scalar('accuracy', compute_accuracy.result(), step=step_num)
        

In [176]:
MODEL_DIR = './tmp/tensorflow/mnist/'
def apply_clean():
    if tf.io.gfile.exists(MODEL_DIR):
        print('Removing existing model dir: {}'.format(MODEL_DIR))
        tf.io.gfile.rmtree(MODEL_DIR)
apply_clean()

checkpoint_dir = os.path.join(MODEL_DIR, 'checkpoints')
print(checkpoint_dir)
os.makedirs(checkpoint_dir,exist_ok=True)
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
print(checkpoint_prefix)
checkpoint = tf.train.Checkpoint(model=model, optimizer=optimizer)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

Removing existing model dir: ./tmp/tensorflow/mnist/
./tmp/tensorflow/mnist/checkpoints
./tmp/tensorflow/mnist/checkpoints\ckpt


In [177]:
NUM_TRAIN_EPOCHS = 1

for i in range(NUM_TRAIN_EPOCHS):
    start = time.time()
    #   with train_summary_writer.as_default():
    train(model, optimizer, train_ds, log_freq=500)
    end = time.time()
    print('Train time for epoch #{} ({:,} total steps): {:.2f} secs'.format(
        i + 1, int(optimizer.iterations), end - start))
    #   with test_summary_writer.as_default():
    #     test(model, test_ds, optimizer.iterations)
    checkpoint.save(checkpoint_prefix)
    print('saved checkpoint.')

export_path = os.path.join(MODEL_DIR, 'export/')
os.makedirs(export_path, exist_ok=True)
tf.saved_model.save(model, export_path)
print('saved SavedModel for exporting.')

step: 8500 loss: 1.8004013 acc: 0.6609
Train time for epoch #1 (8680 total steps): 22.522594213485718
saved checkpoint.
INFO:tensorflow:Assets written to: ./tmp/tensorflow/mnist/export/assets
saved SavedModel for exporting.


In [183]:
print('Train time for epoch #{} ({:,} total steps): {:.2f} secs'.format(
    i + 1, int(optimizer.iterations), end - start))

Train time for epoch #1 (8,680 total steps): 22.52 secs


In [187]:
a= test(model, test_ds,100)

Model test set loss: 1.7753 accuracy: 68.38%
loss: tf.Tensor(1.775289, shape=(), dtype=float32) acc: tf.Tensor(0.6838, shape=(), dtype=float32)


In [212]:
for (images,labels) in test_ds:        
    predictions = model.predict(images)
    for predict in predictions:
        print(predict.argmax(axis=0),end="")
    print()
        

3210919959069015930996659099013136323121139235129963556091953593396930302919329396239933613693191369
6059992199513939999259369905556653510169633131520299551560396659659519932321161516503925011109031692
3611139529959390355592231259133359392291595523039291953321205533910150301999192129359269159292090025
6312902993300319652512930920911215339306331351051315561551399622506563320955911903336162192061952599
2339295031913999192192920391991295999396003020699333239126505666355235596169125919359059910523309906
3932131365392263265919313036319369692162595990023291953993969095096063599339333190331306593309635099
6065515602902231935106962639329322929359100205213369125303929091561193991951239369132333692955511993
1033099992559052100950906193269269313625920621939105931139999990295116991992915535313569991535032512
5399053331335963261360321919252199611295193950931310910355296692535225609292569539930663213229300535
1996029199933956991212232323913903556650636632391125699523399591109199590622315120291269162

6310993920960953503190955239019151395621665012593590123962590123935919533095530596203116913623523595
5531311039269292292923921065395556906305160012395690123995901239325169399091169362095655001993291316
9359959352535505991313962091933916939521566155650123356309012395635901239563096930239305130121320326
9059959531393006663392696319035523901915136121650123956959012395635012356951095663992510699129209339
1523163335902902995090693295605351091692999565262519559350113999931565901239561590123956359012395615
9353293219552321393212591993510069399506153961250999039999196590003916960959139639959339569362209901
2395639901239563190123956309692699559929393520956010653535003915301306235131355920996391626319506999
9623919225331012399590123955901059592613595269395920399913599995552662313293190113500515196009916690
5611012362239562012256392193961329952001665223292161069539630123956999012395695901239569991639901299
3396903995921953952511621396926255029065199192696659992133055503999029919903536012395615901

In [216]:
for (images,labels) in test_ds: 
    for label in labels:
        print("{}".format(label),end="")
    print()

7210414959069015973496654074013134727121174235124463556041957893746430702917329776278473613693141769
6054992194873974449254767905856657810164673171820299551560344654654514472327181818508925011109031642
3611139529459390365572271284173388792241598723044241957728268577918180301994182129759264158292040028
4712402743300319652592930420711215339786561381051315561851794622506563720885411403376162192861952544
2838245031775797192142920491481845988376003026649333239126805666388275896184125919754089910523789406
3952131365742263265489713038319344642182548840023277087447969098046063548339333780821706543809638099
6868578602402231975108462679329822927359180205213767125803724091867743491951739769137833672858511443
1077079448554082108450406173267269314625420621734105431174994840245116471942415538314568941538032512
8344088331735963261360721714242179611248177480731310770355276692835225608292888874930663213229300578
1446029147473988471212232323917403558632676632781175649513347891169144540622315120381267162

6377443928960953887140485239019151748621688012347890123467890123478914533095430846707716913623823895
8871711034264742742927921065348596906308160012345670123478901234725164399097164362098657001743241376
4777984382835805471317962091733916439821864155650123456789012345678901234567896970234385130121320726
4059989531747006663742898714048523901915176121680123456789012345678012356781045663442810649729209339
1523167378402402478070693248605751081672979565262817557350113849451868901234567890123456789012345678
9353293214552321397212891887810067787506157461250799034484186590003716460454138639959378564762209401
2345678901234567890123456789642647554729393820956010653538003415308306278171385420976741626719806949
9623719225378012347890123478901789892613548264345920394973874498582662313273190113507815146004916690
7611012347234567012786397193961724457001668277242161069839630123456789012345678901234567891689901244
3744403875821753852511621386426255028068179192676687492133055803797027917803536012345678901